In [1]:
cd ..

In [2]:
from algorithms.concat import Concat
from algorithms.nmf import NonnegativeMatrixFactorization
from imvc.utils.utils import RandomMultiviewDataset, ConvertToPositive
from metrics.internal import InternalMetrics
from metrics.stability import StabilityMetrics
from metrics.external_validation import ExternalValidation
import warnings

In [3]:
rmd = RandomMultiviewDataset()
mvd = rmd.create_mvd(n_samples = 100, n_features = [10, 50, 100], n_clusters = [3, 4, 2])
sample_views = rmd.add_sample_views(mvd = mvd, p = [0.2, 0.3, 0.5])
imvd = rmd.add_missing_views(mvd = mvd, sample_views = sample_views)
for data in imvd:
    display(data.head())

In [4]:
estimator = Concat(kmeans__n_clusters = 2)
estimator.fit(X = imvd)

In [5]:
pred = estimator.predict(X = imvd)
pred

In [6]:
X = estimator[:-1].fit_transform(imvd)
print(X.shape)

In [7]:
InternalMetrics(X = imvd, estimator= estimator).compute(labels_pred= pred)

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    stability = StabilityMetrics(X = imvd, estimator = estimator, n_times = 3, feature_frac = 0.3, random_state=42)

In [9]:
stability.compute(pred = pred)

In [10]:
apn = stability.apn(pred = pred)
ad = stability.ad(pred = pred)
adm = stability.adm(pred = pred)
fom = stability.fom(pred = pred)
apn, ad, adm, fom

In [11]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    external_validation = ExternalValidation(X = imvd, sample_views = sample_views, estimator = estimator, n_times = 10, sample_frac = 0.5, random_state=42)
external_validation.compute_stability()

In [12]:
estimator = NonnegativeMatrixFactorization()
estimator.fit(X = imvd)

In [107]:
from scipy.stats import chi2_contingency
from joblib import Parallel, delayed
from sklearn.model_selection import ShuffleSplit
import joblib
import pandas as pd


class LabelEvaluation():
    
    def __init__(self, clusters):
        self.clusters = clusters
    
    def compute(self, label, statistic : str, permutation : bool = True, sample_frac = .8, n_times = int(1e4), random_state = None, n_jobs = 1, verbose = 0):
        if statistic == "chi2":
            test = lambda x,y: chi2_contingency(pd.crosstab(x+1, y+1))[1]
        
        permutation_scores = joblib.Parallel(n_jobs = n_jobs, verbose = verbose)(
joblib.delayed(test)(self.clusters.iloc[idx], label.iloc[idx]) for idx,_ in ShuffleSplit(n_splits = n_times, train_size = sample_frac, random_state = random_state).split(self.clusters))

            
        return permutation_scores    


In [108]:
LabelEvaluation(clusters= external_validation.val_preds[0]).compute(label = external_validation.val_labels[0], statistic = "chi2", n_jobs = -1)

In [88]:
import joblib
from sklearn.model_selection import ShuffleSplit

n_splits, sample_frac, n_jobs, verbose, random_state = 10, .8, 2, 0, None
clusters = external_validation.val_preds[0]
label = external_validation.val_labels[0]

permutation_scores = joblib.Parallel(n_jobs=n_jobs, verbose=verbose)(
    joblib.delayed()(clusters.iloc[idx], label.iloc[idx]) for idx,_ in ShuffleSplit(n_splits = n_splits, train_size = sample_frac, random_state = random_state).split(clusters))
permutation_scores

In [ ]:
external_validation.train_pred[0].count()

In [ ]:
x

In [77]:
from scipy.stats import chisquare, permutation_test, chi2_contingency
import numpy as np
import pandas as pd

x = external_validation.val_preds[0] +1
y = external_validation.val_labels[0] +1

result = chi2_contingency(pd.crosstab(x,y))
result

In [55]:
result

In [82]:
chi2_contingency(ct, axis = -1)

In [80]:
res = permutation_test(ct, lambda x: chi2_contingency(x)[0])
res.pvalue

In [69]:
from sklearn.feature_selection import chi2

chi2(x.to_frame(), y)